In [38]:
import sys
from pathlib import Path

# Add project root to path so we can import from eval/ and util/
# In Jupyter notebooks, use the current working directory instead of __file__
notebook_dir = Path.cwd()
# Assuming the notebook is in generators/, go up one level to project root
project_root = notebook_dir.parent if notebook_dir.name == "generators" else notebook_dir
sys.path.insert(0, str(project_root))

from openai import OpenAI
from eval.judge_text import evaluate_text

In [ ]:
import ast
def extract_ideas(text: str) -> str:
    """Extract content between <Post> and </Post> tags."""
    start_tag = "<ideas>"
    end_tag = "</ideas>"
    
    start_idx = text.find(start_tag)
    if start_idx == -1:
        return ""
    
    start_idx += len(start_tag)
    end_idx = text.find(end_tag, start_idx)
    
    if end_idx == -1:
        return ""
    
    return text[start_idx:end_idx].strip()

def extract_block(text, tag):
    import re
    match = re.search(fr"<{tag}>(.*?)</{tag}>", text, re.DOTALL)
    return match.group(1).strip() if match else None

In [40]:
client = OpenAI()

prompt = """<Instructions>Your job is to write the most viral Reddit posts in the "am I the asshole" subreddit.

You will be given a title and content of an already popular post. Please generate a list of tangential, unrelated, potentially surpirsing, and interesting concepts or ideas that may match well with the existing post.  

Respond with a list of the ideas in the following format <ideas>['idea1...', 'idea2...']</ideas>. 

<Title>because I won't tell my wife what my son/her stepson has in savings from my late wife?</Title>

<Post>My wife (40s) and I (40s) have a blended family. I have a 16 year old son with my late wife. My wife has a 13 year old daughter and a 12 year old son with her ex-husband and they share custody. We have a 4 and a 2 year old together.

When my late wife was sick she told me she wanted a percent of the compensation she was issued for her medical misdiagnosis to go to our son's savings. She also had a second savings account she wanted to add to it. This was on top of what the two of us had saved for him before her illness. I honored her wishes and I continued saving for our son's future. This is a promise I made to my late wife that I have stood by. I also promised I would protect it and make sure it was only ever for him, nobody else.

When my present wife and I started seriously dating we talked about what saving would look like for our kids if/when we married. She knew I had always saved as had my late wife. We didn't discuss how much either of us had saved for our existing kids but we agreed to an amount every month that would be saved for all three and this was talked about again when we had children together.

Recently there has been a lot of stress on my wife. Her ex has refused to discuss my stepkids savings and if he pays anything or not. She also feels like we haven't been able to save enough every month like promised because it was just one thing after another. There are month's we've missed because of things beyond our control. Life stuff mostly like things breaking, etc.

A few weeks ago my wife told me we should put all the cards on the table and discuss what all five kids have and decide how we proceed with making sure everyone gets what they need for the future. I told her I would happily discuss what I have saved for my oldest, but I would not be discussing the money left by his mom. She pushed back on it and said all the money should be taken into consideration. I told her we don't know what all the money is. I said our parents could be saving money for the kids' futures, her ex could have money for my stepkids, his parents could have money for my stepkids. We can't say for certainty what everyone will have at the end and it's only fair to discuss what she and I have and can save.

She told me it sounds like I don't trust her. I responded by asking why she wants to know this so badly. By only talking about what we've saved we're on an even footing. She told me if we knew everything we could focus on the kids who will have less a little more. That our younger two are probably going to be the worst off long term and she would be open to combining all the money and dividing it. I told her I was not on board with that and what my son has from his mom is not going to be shared or used for any of the other kids.

We argued about it but I refused to say. She feels like that's wrong when we're married. I told her it's not my money, nor is it her money or money that we can or should be accessing. It's from my late wife to our son and that's all there is to it. My wife said she feels like I'm keeping her in the dark and not trusting her with something important. She said it cuts her ability to fully engage in conversations we need to be able to have and decisions we need to make as a couple.</Post> 
"""

ideas_response = client.responses.create(
    model="gpt-5",
    input=prompt
)

ideas = extract_ideas(ideas_response.output_text)

In [42]:
print(ideas)

['Late wife’s malpractice payout is a structured settlement with an NDA that legally bars disclosure of amounts', 'Funds sit in a UTMA/UGMA account requiring court approval to be used before 18, making “family pot” impossible', 'A third-party trustee (late wife’s sister) controls the legacy money; OP isn’t the trustee and spouse is explicitly excluded', 'Postnup proposal carving out legacy assets vs spouse insisting “marital transparency” includes children’s inheritances', 'Bio grandparents threaten legal action if any of their daughter’s money is shared with non-blood siblings', 'Stepkids’ dad trying to count OP’s son’s inheritance when negotiating child support fairness', 'Factoring company keeps mailing offers to buy the teen’s structured settlement; spouse wants to “at least hear them out”', '529 plans vs custodial accounts: FAFSA impact means secrecy protects the teen’s aid eligibility', 'Spouse previously “borrowed” from her kids’ accounts for a family emergency; trust is shaky b

In [43]:
client = OpenAI()

prompt = f"""<Instructions>Your job is to write the most viral Reddit posts in the "am I the asshole" subreddit.

You will be given a post's title, the post's original content, and a list of unrelated but interesting concepts/ideas. You need to generate two different posts, each between 300-800 words, by incorporating two ideas (one per post) from the list provided into the original post to create three new posts that you think have the best chance to go viral. Each of the new posts should still have the same title as the original post.

IMPORTANT STYLE & CONTENT CONSTRAINTS:
- You MUST NOT copy any full sentences from the original <Post>.
- You MUST NOT reuse distinctive phrases or near-verbatim wording from the original <Post>.
- The new posts should be **substantially different** in wording, sentence structure, and narrative flow.
- You SHOULD preserve the core concept and moral dilemma of the original, but you are free to:
  - The setting, the background, the backstory. 
  - Change the pacing and structure of how events are revealed.
  - Add or modify specific scenes, examples, or conversations (as long as they stay plausible).
  - Adjust tone, emotional emphasis, and narrative voice while staying in first-person AITA style.
- Think of this as writing a **new story inspired by the same situation** and the new idea, not paraphrasing or lightly editing the original.

The resulting new stories must:
- Be plausible and realistic
- Be highly relatable
- Contain drama, conflict, and no easy answer
- Encourage a lot of comments and debate

For each of the three posts, you MUST also:
1. Explicitly state which single idea from the <ideas> list you chose to incorporate.
2. Explain briefly why you believe using that idea makes the post more likely to go viral (e.g., more moral ambiguity, higher stakes, more intense conflict, unusual twist, etc.).

Output format (VERY IMPORTANT):
- Put the full text of the three rewritten posts inside:
  - <NewPost1>...</NewPost1>
  - <NewPost2>...</NewPost2>
  - <NewPost3>...</NewPost3>

- For each post, ALSO output:
  - The idea used, copied exactly as it appears in the <ideas> list, inside:
    - <NewPost1Idea>...</NewPost1Idea>
    - <NewPost2Idea>...</NewPost2Idea>
    - <NewPost3Idea>...</NewPost3Idea>

  - A short explanation (2–5 sentences) of why that idea increases the post's viral potential, inside:
    - <NewPost1WhyViral>...</NewPost1WhyViral>
    - <NewPost2WhyViral>...</NewPost2WhyViral>
    - <NewPost3WhyViral>...</NewPost3WhyViral>

Do NOT include any extra commentary outside these tags. Do NOT output JSON. Only use the tags described above.</Instructions>
<Title>because I won't tell my wife what my son/her stepson has in savings from my late wife?</Title>

<Post>My wife (40s) and I (40s) have a blended family. I have a 16 year old son with my late wife. My wife has a 13 year old daughter and a 12 year old son with her ex-husband and they share custody. We have a 4 and a 2 year old together.

When my late wife was sick she told me she wanted a percent of the compensation she was issued for her medical misdiagnosis to go to our son's savings. She also had a second savings account she wanted to add to it. This was on top of what the two of us had saved for him before her illness. I honored her wishes and I continued saving for our son's future. This is a promise I made to my late wife that I have stood by. I also promised I would protect it and make sure it was only ever for him, nobody else.

When my present wife and I started seriously dating we talked about what saving would look like for our kids if/when we married. She knew I had always saved as had my late wife. We didn't discuss how much either of us had saved for our existing kids but we agreed to an amount every month that would be saved for all three and this was talked about again when we had children together.

Recently there has been a lot of stress on my wife. Her ex has refused to discuss my stepkids savings and if he pays anything or not. She also feels like we haven't been able to save enough every month like promised because it was just one thing after another. There are month's we've missed because of things beyond our control. Life stuff mostly like things breaking, etc.

A few weeks ago my wife told me we should put all the cards on the table and discuss what all five kids have and decide how we proceed with making sure everyone gets what they need for the future. I told her I would happily discuss what I have saved for my oldest, but I would not be discussing the money left by his mom. She pushed back on it and said all the money should be taken into consideration. I told her we don't know what all the money is. I said our parents could be saving money for the kids' futures, her ex could have money for my stepkids, his parents could have money for my stepkids. We can't say for certainty what everyone will have at the end and it's only fair to discuss what she and I have and can save.

She told me it sounds like I don't trust her. I responded by asking why she wants to know this so badly. By only talking about what we've saved we're on an even footing. She told me if we knew everything we could focus on the kids who will have less a little more. That our younger two are probably going to be the worst off long term and she would be open to combining all the money and dividing it. I told her I was not on board with that and what my son has from his mom is not going to be shared or used for any of the other kids.

We argued about it but I refused to say. She feels like that's wrong when we're married. I told her it's not my money, nor is it her money or money that we can or should be accessing. It's from my late wife to our son and that's all there is to it. My wife said she feels like I'm keeping her in the dark and not trusting her with something important. She said it cuts her ability to fully engage in conversations we need to be able to have and decisions we need to make as a couple.</Post> 

<ideas>{ideas}</ideas>
"""

response = client.responses.create(
    model="gpt-5",
    input=prompt
)


In [49]:
print(extract_block(response.output_text, "NewPost2Idea"))

Spouse previously “borrowed” from her kids’ accounts for a family emergency; trust is shaky because of that history


In [52]:
import time
print(time.time())
evaluation = evaluate_text(extract_block(response.output_text, "NewPost2"))
print(time.time())

1763243910.044862
[1763243910.069] Starting judge 0
[1763243910.070] Starting judge 1
[1763243910.070] Starting judge 2
[1763243910.071] Starting judge 3
[1763243910.071] Starting judge 4
[1763243910.072] Starting judge 5
[1763243910.072] Starting judge 6
[1763243910.072] Starting judge 7
[1763243910.074] Starting judge 8
[1763243914.353] Completed judge 2 in 4.28s
[1763243916.022] Completed judge 3 in 5.95s
[1763243916.508] Completed judge 0 in 6.44s
[1763243916.511] Completed judge 8 in 6.44s
[1763243916.564] Completed judge 5 in 6.49s
[1763243916.747] Completed judge 4 in 6.68s
[1763243917.119] Completed judge 6 in 7.05s
[1763243917.639] Completed judge 7 in 7.57s
[1763243918.055] Completed judge 1 in 7.99s
1763243918.057398


In [53]:
print(evaluation['score'])

judge_list = list(evaluation['judges'].values())
for j in judge_list:
    print(j['question'], ' ', j['answer'])

0.9
Does the text demonstrate moral complexity where reasonable people could disagree about who is right?   True
Does the narrator describe their own emotional journey or growth rather than just seeking validation?   True
Does the below text have a compelling hook? A compelling hook should demonstrate several of these qualities: 
- Emotional intensity: Creates strong feelings (anger, sympathy, shock, curiosity, fear)
- Gossip-worthiness: Presents a naturally discussable situation that people would want to talk about
- Clear conflict: Shows an obvious problem, tension, or injustice
- Relatable themes: Involves universal experiences like relationships, justice/injustice, embarrassment, betrayal, family dynamics, or workplace situations
- Character positioning: Establishes the character as either wronged/righteous OR in a ridiculous/embarrassing situation
- Immediate engagement: Hooks the reader within the first 5 seconds (opening sentences)
- Cliffhanger: Ends on a suspenseful note that 

because I won't tell my wife what my son/her stepson has in savings from my late wife?

I’ll own that I’m not the most relaxed person about money. But I have reasons. My late wife asked me to safeguard a chunk of her settlement and her personal savings for our son. I promised her I would make sure it was there for him when he needed it, no strings, no “loans.”

Fast forward: I married my current wife, who has two kids. We agreed to contribute monthly to all the kids’ futures. We’ve been trying, but life has been expensive. Recently she said, “Let’s stop guessing and put actual numbers on the table for all five kids.” I agreed to go through everything we’ve contributed as a couple and what I’ve put into our joint children’s accounts. But I declined to disclose the total that came from my late wife and the exact number my son has.

Why I’m stuck: two years ago our house flooded and we were scrambling. My wife pulled money out of her two kids’ accounts “just until the insurance check cleared.” I didn’t sign off; she did it while I was traveling. She paid back most of it, but not all, and her ex was furious when he found out. It created a huge blowup, therapy, rules, the whole thing. We agreed after that: no touching kids’ money and no using the kids’ accounts to solve adult emergencies.

So when she tells me now that knowing my son’s number would help us be “more fair,” my brain goes straight to that day. I don’t think she’s a thief. I think she’s a fixer under pressure who will rationalize “borrowing from future us” when things get scary. And once the number is out there, it can’t be untold. It changes the dynamic for my son too—he’s already anxious about people expecting him to “share.”

She says I’m punishing her forever for a mistake and treating her like an outsider. I told her it’s not about punishment; it’s about a boundary I set long before we married, one I’m responsible for keeping. I also pointed out that there are unknowns on her side too—maybe her ex has savings for their kids, maybe grandparents are quietly contributing. It’s not just my son who might have “extra.” The only ethical way I see this is to plan using the accounts we jointly control and keep contributing evenly going forward.

Now it’s icy in our house. She says I’m withholding critical information that affects family planning. I think telling her would just load a target on my son’s back for years—every crisis becomes a referendum on whether he should “help the family.” I suggested a neutral financial mediator and even offered to let her see the attorney who set up my son’s accounts without the dollar figure disclosed, so she understands the restrictions. She said that’s “lawyering our marriage.”

So Reddit, am I the asshole for refusing to reveal the amount, given the history and the promise I made?

In [1]:
from pathlib import Path
from tony_script import generate_and_save_for_file
from tqdm import tqdm

def load_title_and_content(path):
    text = Path(path).read_text(encoding="utf-8").strip()

    lines = text.splitlines()

    # First line starts with "Title:"
    if not lines[0].lower().startswith("title:"):
        raise ValueError("File does not start with 'Title:' line.")

    title = lines[0].split(":", 1)[1].strip()   # everything after Title:
    content = "\n".join(lines[1:]).strip()      # rest of file

    return title, content



folder = Path("../texts/good_samples")

files = [f for f in folder.iterdir() if f.is_file() and not f.name.startswith(".")]

for file in tqdm(files, desc="Generating post", unit="file"):
    title, content = load_title_and_content(file)
    generate_and_save_for_file(title, file)


Generating post:   0%|          | 0/12 [00:00<?, ?file/s]

[1763269867.757] Starting judge 0[1763269867.757] Starting judge 1

[1763269867.759] Starting judge 2
[1763269867.759] Starting judge 3
[1763269867.760] Starting judge 4
[1763269867.760] Starting judge 5
[1763269867.760] Starting judge 6
[1763269867.761] Starting judge 7
[1763269867.761] Starting judge 8
[1763269873.453] Completed judge 6 in 5.69s
[1763269873.972] Completed judge 1 in 6.22s
[1763269874.109] Completed judge 5 in 6.35s
[1763269874.223] Completed judge 2 in 6.46s
[1763269874.671] Completed judge 4 in 6.91s
[1763269876.727] Completed judge 0 in 8.97s
[1763269877.343] Completed judge 8 in 9.58s
[1763269879.596] Completed judge 7 in 11.84s
[1763269879.901] Completed judge 3 in 12.14s
[1763269879.937] Starting judge 0
[1763269879.937] Starting judge 1
[1763269879.938] Starting judge 2
[1763269879.938] Starting judge 3
[1763269879.939] Starting judge 4
[1763269879.942] Starting judge 5
[1763269879.942] Starting judge 6
[1763269879.942] Starting judge 7
[1763269879.943] Startin

Generating post:   8%|▊         | 1/12 [02:25<26:40, 145.52s/file]

[1763269896.025] Completed judge 1 in 16.09s
[1763270089.394] Starting judge 0[1763270089.395] Starting judge 1

[1763270089.396] Starting judge 2
[1763270089.396] Starting judge 3
[1763270089.397] Starting judge 4
[1763270089.397] Starting judge 5
[1763270089.398] Starting judge 6
[1763270089.398] Starting judge 7
[1763270089.399] Starting judge 8
[1763270093.779] Completed judge 5 in 4.38s
[1763270093.948] Completed judge 2 in 4.55s
[1763270094.428] Completed judge 6 in 5.03s
[1763270095.453] Completed judge 3 in 6.06s
[1763270095.536] Completed judge 7 in 6.14s
[1763270096.742] Completed judge 1 in 7.35s
[1763270097.015] Completed judge 4 in 7.62s
[1763270099.549] Completed judge 0 in 10.15s
[1763270111.294] Completed judge 8 in 21.89s
[1763270111.329] Starting judge 0
[1763270111.329] Starting judge 1
[1763270111.330] Starting judge 2
[1763270111.330] Starting judge 3
[1763270111.331] Starting judge 4
[1763270111.331] Starting judge 5
[1763270111.331] Starting judge 6
[1763270111.3

Generating post:  17%|█▋        | 2/12 [06:13<32:17, 193.74s/file]

...

[1763270809.702] Completed judge 4 in 17.18s
[1763270865.719] Starting judge 0[1763270865.719] Starting judge 1

[1763270865.720] Starting judge 2
[1763270865.721] Starting judge 3
[1763270865.721] Starting judge 4
[1763270865.722] Starting judge 5
[1763270865.728] Starting judge 6
[1763270865.729] Starting judge 7
[1763270865.729] Starting judge 8
[1763270869.557] Completed judge 6 in 3.83s
[1763270870.006] Completed judge 5 in 4.28s
[1763270871.030] Completed judge 0 in 5.31s
[1763270871.237] Completed judge 2 in 5.52s
[1763270871.747] Completed judge 1 in 6.03s
[1763270872.892] Completed judge 3 in 7.17s
[1763270875.126] Completed judge 7 in 9.40s
[1763270875.128] Completed judge 4 in 9.41s
[1763270876.522] Completed judge 8 in 10.79s
[1763270876.557] Starting judge 0
[1763270876.558] Starting judge 1
[1763270876.558] Starting judge 2
[1763270876.559] Starting judge 3
[1763270876.559] Starting judge 4
[1763270876.560] Starting judge 5
[1763270876.560] Starting judge 6
[1763270876.56

Generating post:  92%|█████████▏| 11/12 [18:56<01:24, 84.71s/file]

[1763270887.273] Completed judge 7 in 10.71s
[1763270960.075] Starting judge 0
[1763270960.076] Starting judge 1
[1763270960.077] Starting judge 2
[1763270960.078] Starting judge 3
[1763270960.078] Starting judge 4
[1763270960.079] Starting judge 5
[1763270960.079] Starting judge 6
[1763270960.079] Starting judge 7
[1763270960.079] Starting judge 8
[1763270964.627] Completed judge 6 in 4.55s
[1763270964.632] Completed judge 7 in 4.55s
[1763270964.646] Completed judge 2 in 4.57s
[1763270964.848] Completed judge 0 in 4.77s
[1763270966.263] Completed judge 5 in 6.18s
[1763270968.102] Completed judge 8 in 8.02s
[1763270968.619] Completed judge 3 in 8.54s
[1763270975.172] Completed judge 4 in 15.09s
[1763270977.115] Completed judge 1 in 17.04s
[1763270977.148] Starting judge 0
[1763270977.149] Starting judge 1
[1763270977.149] Starting judge 2
[1763270977.149] Starting judge 3
[1763270977.150] Starting judge 4
[1763270977.150] Starting judge 5
[1763270977.151] Starting judge 6
[1763270977.1

Generating post: 100%|██████████| 12/12 [20:40<00:00, 103.37s/file]

[1763270991.003] Completed judge 8 in 13.85s
